In [62]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from src.utils.UsefulPaths import Paths
from src.utils.SpacyUtils import SpacyUtil
import numpy as np
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [63]:
paths = Paths()

In [64]:
df_subsectors = pd.read_csv(os.path.join(paths.data_processed, 'subsector.csv'))
df_subsectors = df_subsectors.fillna('')

df_subsectors = df_subsectors[:-1]

In [65]:
df_subsectors

,subsector,definition,keywords,old_name,does_include,does_not_include,definition_preprocessed,keywords_preprocessed,does_include_preprocessed,does_not_include_preprocessed,token_definition,token_keywords,token_does_include,token_does_not_include
0,Artificial Intelligence; Big Data and Analytics,Artificial Intelligence companies offer produc...,"Automated intelligence, assisted intelligence,...","Artificial Intelligence, Big Data and Analytics",,,artificial intelligence company offer product ...,automated intelligence assist intelligence aug...,,,73,62,0,0
1,Advanced Manufacturing and Robotics,"In comparison to Traditional Manufacturing, Ad...","3d printing, industrial IoT, internet of thin...",Advanced Manufacturing and Robotics,"Autonomous driving, Industrial robots, Industr...",Traditional manufacturing machines,in comparison to traditional manufacturing adv...,3d printing industrial iot internet of thing...,autonomous driving industrial robot industrial...,traditional manufacturing machine,29,21,27,3
2,Clean Technology,Cleantech or clean technology is an umbrella t...,"Clean energy, and other forms of environmental...",Cleantech,,"oil and gas, petrochemicals",cleantech or clean technology be an umbrella t...,clean energy and other form of environmental a...,,oil and gas petrochemical,67,30,0,4
3,Financial Technology,Describes a business that aims at providing fi...,"Insurance Tech, Risk Management, Trading, Port...",Fintech,,"Brick & Mortar banks, Old brick and mortar Ins...",describe a business that aim at provide financ...,insurance tech risk management trading portfol...,,brick mortar bank old brick and mortar insuran...,65,21,0,9
4,Blockchain,Companies that develop applications using bloc...,"Distributed ledgers, Digital Mining,Cryptocurr...",Blockchain,,,company that develop application use blockchai...,distribute ledger digital mining cryptocurrenc...,,,52,26,0,0
5,Cybersecurity,"Cybersecurity is the body of technologies, pro...","cyber security, network security, data securit...",Cybersecurity,"Application security, Information security, Ne...",,cybersecurity be the body of technology proces...,cyber security network security data security ...,application security information security netw...,,42,15,17,0
6,Agriculture Technology,Technologies to help the agriculture industry ...,"agritech, crop optimization, farming automatio...",Agtech,Precision agriculture - PA is an approach to f...,"Farms, Vineyards, Coffee roasters, Beverages",technology to help the agriculture industry to...,agritech crop optimization farming automation ...,precision agriculture pa be an approach to far...,farms vineyards coffee roaster beverages,84,9,62,5
7,New Food,New Food includes technologies that can be lev...,"artificial meat, Alternative protein, Plant-ba...",New Food,,"alt- proteins based skincare products, cannabi...",new food include technology that can be levera...,artificial meat alternative protein plant base...,,alt- protein base skincare product cannabis re...,63,52,0,27
8,Advertising Technology,Advertising technology - different types of an...,"Conversion/optimization, Email marketing, Mobi...",Adtech,,Companies whose products and services are not ...,advertising technology different type of analy...,conversion optimization email marketing mobile...,,company whose product and service be not focus...,50,38,0,54
9,Blue Economy,"Blue economy is the ""sustainable use of ocean ...","ocean sustainability, aquaculture, seafloor ma...",Blue Economy,,,blue economy be the sustainable use of ocean r...,ocean sustainability aquaculture seafloor mapp...,,,53,27,0,0


In [66]:
all_subsectors_names = df_subsectors['subsector'].tolist()
all_subsectors_names = ''.join(all_subsectors_names)
all_subsectors_names = all_subsectors_names.replace(';', ',')
all_subsectors_names

'Artificial Intelligence, Big Data and AnalyticsAdvanced Manufacturing and RoboticsClean TechnologyFinancial TechnologyBlockchainCybersecurityAgriculture TechnologyNew FoodAdvertising TechnologyBlue EconomyDigital MediaGamingAugmented Reality, Virtual RealityEducational TechnologyIndustry 4.0Biopharmaceutical, BiotechonologyMedical Technology, Medical devices'

In [67]:
df_abstract_patents = pd.read_csv(os.path.join(paths.data_processed, 'abstract_patents_sliced_15000.csv'))

In [68]:
slice_number = 100
df_sliced = df_abstract_patents.loc[:slice_number, ['abstract']].copy()

df_sliced['sentence_abstract'] = 'What subsector definition best describe this patent abstract "' + df_sliced['abstract'] + '"'
# df_sliced['sentence_keywords'] = 'What keywords best describe this patent abstract "' + df_sliced['abstract'] + '"'

spacy_util = SpacyUtil(model='en_core_web_sm', lemma=True, remove_stopwords=True, lower=True, remove_numbers=False)

df_sliced['sentence_abstract'] = df_sliced['sentence_abstract'].apply(spacy_util.preprocess_text)
df_sliced['sentence_keywords'] = df_sliced['abstract'].apply(lambda text: spacy_util.extract_keywords(text=text, max_keywords=100, use_noun=True, use_adj=True, use_verb=True))
df_sliced['sentence_keywords'] = 'This keywords  "' + df_sliced['sentence_keywords'] + '" describe which of these types of subsectors ' + all_subsectors_names
# df_sliced['sentence_does_not_included'] = 'This keywords  "' + df_sliced['abstract'] + '" describe which of these types of subsectors ' + all_subsectors_names


df_sliced

,abstract,sentence_abstract,sentence_keywords
0,"A method, and a mobile device adapted thereto,...",subsector definition well describe patent abst...,"This keywords ""user, input, instruction, meth..."
1,A dumbbell with a selectable number of weight ...,subsector definition well describe patent abst...,"This keywords ""weight, disk, handle, connect,..."
2,A virtual assistant AI system that may be conn...,subsector definition well describe patent abst...,"This keywords ""account, change, virtual, assi..."
3,Connector assemblies that are separate from me...,subsector definition well describe patent abst...,"This keywords ""lead, connector, extension, as..."
4,"An LED tube lamp, comprising a lamp tube, whic...",subsector definition well describe patent abst...,"This keywords ""light, portion, lead, include,..."
...,...,...,...
96,A method for controlling a crane component of ...,subsector definition well describe patent abst...,"This keywords ""crane, component, forbidden, v..."
97,A camera head is provided that includes a firs...,subsector definition well describe patent abst...,"This keywords ""lens, group, include, second, ..."
98,An organic light emitting diode display includ...,subsector definition well describe patent abst...,"This keywords ""light, emit, red, emission, re..."
99,An image processing device includes a receptio...,subsector definition well describe patent abst...,"This keywords ""limit, information, section, r..."


In [69]:
# all-mpnet-base-v2
# msmarco-distilbert-dot-v5
# distilbert-base-uncased
# all-MiniLM-L6-v2
# all-MiniLM-L12-v2
# microsoft/mpnet-base
embedder = SentenceTransformer('all-mpnet-base-v2')

2023-11-08 16:57:54,427 - INFO - Load pretrained SentenceTransformer: all-mpnet-base-v2
2023-11-08 16:57:54,926 - INFO - Use pytorch device: cpu


In [70]:
max_tokens = embedder.get_max_seq_length()
print(f"Maximum tokens allowed: {max_tokens}")

Maximum tokens allowed: 384


In [71]:
def get_corpus_id_from_element(semantic_list, i):
    return semantic_list[i].get('corpus_id')

In [72]:
df_semantic_results = pd.DataFrame({'abstract': df_sliced.abstract})

In [73]:
# Definition semantic search
sentences = list(df_sliced.sentence_abstract)
sentences_embeddings = embedder.encode(sentences, normalize_embeddings=True)

corpus = list(df_subsectors['definition_preprocessed'])
corpus_embeddings = embedder.encode(corpus, normalize_embeddings=True)

# Because using normalized vectors, dot_score is a better function instead of cos_sim
df = pd.DataFrame({'definition_preprocessed': util.semantic_search(sentences_embeddings, corpus_embeddings, top_k=2, score_function=util.dot_score)})
    
df['highest_0'] = df['definition_preprocessed'].apply(lambda x: get_corpus_id_from_element(x, 0))
df['highest_1'] = df['definition_preprocessed'].apply(lambda x: get_corpus_id_from_element(x, 1))
    
corpus_id_to_subsector = pd.Series(df_subsectors.subsector.values, index=df_subsectors.index).to_dict()
    
df[f'definition_1'] = df['highest_0'].map(corpus_id_to_subsector)
df[f'definition_2'] = df['highest_1'].map(corpus_id_to_subsector)
    
df = df.drop(['highest_0', 'highest_1'], axis=1)
    
df_semantic_results = pd.concat([df_semantic_results, df], axis=1)

df

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,definition_preprocessed,definition_1,definition_2
0,"[{'corpus_id': 4, 'score': 0.37911149859428406...",Blockchain,Medical Technology; Medical devices
1,"[{'corpus_id': 16, 'score': 0.3615938723087311...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics
2,"[{'corpus_id': 0, 'score': 0.5747168064117432}...",Artificial Intelligence; Big Data and Analytics,Augmented Reality; Virtual Reality
3,"[{'corpus_id': 16, 'score': 0.4684583246707916...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics
4,"[{'corpus_id': 16, 'score': 0.3425365686416626...",Medical Technology; Medical devices,Blockchain
...,...,...,...
96,"[{'corpus_id': 16, 'score': 0.4293273687362671...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics
97,"[{'corpus_id': 16, 'score': 0.4674776792526245...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics
98,"[{'corpus_id': 16, 'score': 0.3556002676486969...",Medical Technology; Medical devices,Artificial Intelligence; Big Data and Analytics
99,"[{'corpus_id': 16, 'score': 0.3983959257602691...",Medical Technology; Medical devices,Digital Media


In [74]:
# Keywords semantic search
sentences = list(df_sliced.sentence_keywords)
sentences_embeddings = embedder.encode(sentences, normalize_embeddings=True)

corpus = list(df_subsectors['keywords_preprocessed'])
corpus_embeddings = embedder.encode(corpus, normalize_embeddings=True)

# Because using normalized vectors, dot_score is a better function instead of cos_sim
df = pd.DataFrame({'keywords_preprocessed': util.semantic_search(sentences_embeddings, corpus_embeddings, top_k=2, score_function=util.dot_score)})
    
df['highest_0'] = df['keywords_preprocessed'].apply(lambda x: get_corpus_id_from_element(x, 0))
df['highest_1'] = df['keywords_preprocessed'].apply(lambda x: get_corpus_id_from_element(x, 1))
    
corpus_id_to_subsector = pd.Series(df_subsectors.subsector.values, index=df_subsectors.index).to_dict()
    
df[f'keywords_1'] = df['highest_0'].map(corpus_id_to_subsector)
df[f'keywords_2'] = df['highest_1'].map(corpus_id_to_subsector)
    
df = df.drop(['highest_0', 'highest_1'], axis=1)
    
df_semantic_results = pd.concat([df_semantic_results, df], axis=1)

df

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,keywords_preprocessed,keywords_1,keywords_2
0,"[{'corpus_id': 3, 'score': 0.5950531959533691}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
1,"[{'corpus_id': 3, 'score': 0.5325780510902405}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
2,"[{'corpus_id': 3, 'score': 0.6411076784133911}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
3,"[{'corpus_id': 3, 'score': 0.5191670060157776}...",Financial Technology,Advanced Manufacturing and Robotics
4,"[{'corpus_id': 3, 'score': 0.5101721882820129}...",Financial Technology,Advanced Manufacturing and Robotics
...,...,...,...
96,"[{'corpus_id': 3, 'score': 0.5295559167861938}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
97,"[{'corpus_id': 3, 'score': 0.5366716384887695}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
98,"[{'corpus_id': 1, 'score': 0.46616318821907043...",Advanced Manufacturing and Robotics,Financial Technology
99,"[{'corpus_id': 3, 'score': 0.5473448634147644}...",Financial Technology,Artificial Intelligence; Big Data and Analytics


In [75]:
df_semantic_results

,abstract,definition_preprocessed,definition_1,definition_2,keywords_preprocessed,keywords_1,keywords_2
0,"A method, and a mobile device adapted thereto,...","[{'corpus_id': 4, 'score': 0.37911149859428406...",Blockchain,Medical Technology; Medical devices,"[{'corpus_id': 3, 'score': 0.5950531959533691}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
1,A dumbbell with a selectable number of weight ...,"[{'corpus_id': 16, 'score': 0.3615938723087311...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics,"[{'corpus_id': 3, 'score': 0.5325780510902405}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
2,A virtual assistant AI system that may be conn...,"[{'corpus_id': 0, 'score': 0.5747168064117432}...",Artificial Intelligence; Big Data and Analytics,Augmented Reality; Virtual Reality,"[{'corpus_id': 3, 'score': 0.6411076784133911}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
3,Connector assemblies that are separate from me...,"[{'corpus_id': 16, 'score': 0.4684583246707916...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics,"[{'corpus_id': 3, 'score': 0.5191670060157776}...",Financial Technology,Advanced Manufacturing and Robotics
4,"An LED tube lamp, comprising a lamp tube, whic...","[{'corpus_id': 16, 'score': 0.3425365686416626...",Medical Technology; Medical devices,Blockchain,"[{'corpus_id': 3, 'score': 0.5101721882820129}...",Financial Technology,Advanced Manufacturing and Robotics
...,...,...,...,...,...,...,...
96,A method for controlling a crane component of ...,"[{'corpus_id': 16, 'score': 0.4293273687362671...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics,"[{'corpus_id': 3, 'score': 0.5295559167861938}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
97,A camera head is provided that includes a firs...,"[{'corpus_id': 16, 'score': 0.4674776792526245...",Medical Technology; Medical devices,Advanced Manufacturing and Robotics,"[{'corpus_id': 3, 'score': 0.5366716384887695}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
98,An organic light emitting diode display includ...,"[{'corpus_id': 16, 'score': 0.3556002676486969...",Medical Technology; Medical devices,Artificial Intelligence; Big Data and Analytics,"[{'corpus_id': 1, 'score': 0.46616318821907043...",Advanced Manufacturing and Robotics,Financial Technology
99,An image processing device includes a receptio...,"[{'corpus_id': 16, 'score': 0.3983959257602691...",Medical Technology; Medical devices,Digital Media,"[{'corpus_id': 3, 'score': 0.5473448634147644}...",Financial Technology,Artificial Intelligence; Big Data and Analytics
